This code loads the data file using langchain and passes it through the OpenAI API for emebdding.  It is then stored in a FAISS index.    The emebedding time depends on the file passed, but is usually <15 minutes.    The data file is created in a seperate script that scrapes the Wiki and Slack Channels.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import SlackDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# OpenAI API Key  (Need to change to an envo var)
key = ''

# Load and split the Dataset
loader1 = PyPDFLoader("path/to/file/All_Data_PDF.pdf")
pages1 = loader1.load_and_split()

#loader2 = SlackDirectoryLoader('C:/Users/avido/OneDrive/Desktop/Chamor/TAG Support Slack export Sep 21 2016 - Sep 18 2023.zip')
#pages2=loader2.load_and_split()

pages=pages1

# Create Index and Embeddings
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=key))

print('Indexed!')

This script creates a Flask server which can then be accessed with a redirect URL from ngrok.  The Bot ID and Channel ID are currently hard coded for testing purposes.   There are a few prints and various, crucial checkpoints.   The script also checks the message time-stamp.  This was included due to an issue with the Bot looping through all mentions and responding.

In [ ]:
import os
import time
import slack
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from datetime import datetime, timedelta
#!pip install slackeventsapi

from flask import Flask
import slack
from slackeventsapi import SlackEventAdapter
from slack_sdk import WebClient
import os

# Initialize the Slack API client with the hardcoded token- Need to switch to envo
client = WebClient(token="")

app=Flask(__name__)

#Assign a URL extension and pass the Client Secret tio authenticate
slack_event_adapter=SlackEventAdapter('','/slack/events',app)


#For future use- retrives bot id
bot_id=client.api_call("auth.test")['user_id']


# Initialize the ChatOpenAI model
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0, openai_api_key=key)

# Initialize the MultiQueryRetriever
retriever = MultiQueryRetriever.from_llm(retriever=faiss_index.as_retriever(), llm=llm)

# Create a QA chain
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)


# Define a function to process messages
def process_message(message):
    
    #Checkpoint
    print('here')
    # Split the message into smaller chunks to account for token limits
    max_tokens = 4000  # Under context length supported by the model
    chunked_message = ""
    chunked_messages = []

    for word in message.split():
        if len(chunked_message) + len(word) + 1 >= max_tokens:
            if chunked_message:
                chunked_messages.append(chunked_message)
            chunked_message = ""
        if chunked_message:
            chunked_message += " "
        chunked_message += word

    if chunked_message:
        chunked_messages.append(chunked_message)

    response = ""
    for chunked_message in chunked_messages:
        result = qa_chain({'query': chunked_message})
        answer = result['result']
        response += answer + " "

    return response


@slack_event_adapter.on('message')
def send_message(payload):
    event=payload.get('event',{})
    channel_id=''   #Hard-coded to only allow messages from bot-testing channel
    user_id=event.get('user')
    text=event.get('text')
    message_ts = float(event.get('ts'))  # Convert timestamp to float
    
    # Define a threshold for considering messages (needed to prevent bot responding in a loop to all previous messages)
    threshold_time = datetime.now() - timedelta(seconds=2)
    threshold_ts = threshold_time.timestamp()

    
    if user_id !=bot_id:  #check that message is not from Bot (not really needed as only listens for @bot, but may need)
       
        if f'<@{bot_id}>' in event['text'] and message_ts >= threshold_ts:
            #checkpoint
            print('rerun')
            time.sleep(2)
            
            #pass text to function to get answer
            response = process_message(text) 
            
            #send response to slack
            client.chat_postMessage(channel=channel_id, text=response)
            return
           
        
    
    time.sleep(1)  # Add a 1-second delay between API calls to avoid rate limiting     

        
# Start listening for messages from Slack
if __name__ == "__main__":
    #Run flask.  Using debug=true throws an error- need to check
    app.run()     
    
    